In [19]:
# !git clone https://github.com/HIT-SCIR/ELMoForManyLangs.git
# !pip install overrides
# !pip install ELMoForManyLangs
# !wget https://3f7xrg.bl.files.1drv.com/y4mzCOdmCDMRHErLHsESWkD0rmmY1j9ca3CfhfCpv6poE3j-0dZd9HmKVC3k0LWif3I2XgyC1tErV8SrVr1mJNVNHYPmU_qqNvvZVBhOijBfsdwWaYVs6Zd4QzsC4HaljGNbTWwtnQ-JrWog9EB0DbblDlKlNBYxcroYpLW9_qrHX7Ub2XEnYVcZ1gqMptzr3Us9Jj66IdrWRLoaYK_FJWiRQ -O dataset.zip
# !unzip -P Ymj26Uv5 dataset.zip

--2024-10-08 09:53:10--  https://3f7xrg.bl.files.1drv.com/y4mzCOdmCDMRHErLHsESWkD0rmmY1j9ca3CfhfCpv6poE3j-0dZd9HmKVC3k0LWif3I2XgyC1tErV8SrVr1mJNVNHYPmU_qqNvvZVBhOijBfsdwWaYVs6Zd4QzsC4HaljGNbTWwtnQ-JrWog9EB0DbblDlKlNBYxcroYpLW9_qrHX7Ub2XEnYVcZ1gqMptzr3Us9Jj66IdrWRLoaYK_FJWiRQ
Resolving 3f7xrg.bl.files.1drv.com (3f7xrg.bl.files.1drv.com)... 13.107.42.12
Connecting to 3f7xrg.bl.files.1drv.com (3f7xrg.bl.files.1drv.com)|13.107.42.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 543548748 (518M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>] 518.37M  35.1MB/s    in 15s     

2024-10-08 09:53:25 (35.5 MB/s) - ‘dataset.zip’ saved [543548748/543548748]

Archive:  dataset.zip
   creating: EATD-Corpus/
  inflating: EATD-Corpus/.DS_Store   
   creating: EATD-Corpus/t_1/
 extracting: EATD-Corpus/t_1/label.txt  
  inflating: EATD-Corpus/t_1/negative.txt  
  inflating: EATD-Corpus/t_1/negative.wav  
  inflating: EATD-Corpus/t_1/

In [8]:
import numpy as np
import jieba
import os
from elmoformanylangs import Embedder

elmo = Embedder('./import_files/zhs.model')
answers = {}
text_features = []
text_clf_targets = []
text_reg_targets = []

2024-10-19 14:36:06,848 WARNING: Could not find config.  Trying ./import_files/zhs.model\cnn_50_100_512_4096_sample.json
2024-10-19 14:36:06,849 WARNING: Could not find config.  Trying c:\Users\meena\anaconda3\Lib\site-packages\elmoformanylangs\configs\cnn_50_100_512_4096_sample.json
2024-10-19 14:36:06,863 INFO: char embedding size: 6169
2024-10-19 14:36:07,082 INFO: word embedding size: 71222
2024-10-19 14:36:09,394 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(71222, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(6169, 50, padding_idx=6166)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      

In [9]:
def extract_features(text_features, text_clf_targets, text_reg_targets):
    base_path = os.path.join(os.getcwd(), 'EATD-Corpus')
    topics = ['positive', 'neutral', 'negative']

    indx=0
    for dir_name in os.listdir(base_path):
        indx = indx+1

        dir_path = os.path.join(base_path, dir_name)
        if not os.path.isdir(dir_path):
            continue
        answers[indx] = []

        for topic in topics:
            file_path = os.path.join(dir_path, f'{topic}.txt')

            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    lines = f.readlines()[0]
                    seg_text_iter = jieba.cut(lines, cut_all=False)
                    # answers[i].append(list(seg_text_iter))
                    answers[indx].append(item for item in seg_text_iter)
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                continue

        label_path = os.path.join(dir_path, 'new_label.txt')
        try:
            with open(label_path, 'r', encoding='utf-8') as f:
                target = float(f.readline())

            text_clf_targets.append(1 if target >= 53 else 0)
            text_reg_targets.append(target)
            text_features.append([np.array(item).mean(axis=0) for item in elmo.sents2elmo(answers[indx])])

        except FileNotFoundError:
            print(f"Label file not found: {label_path}")
        except ValueError:
            print(f"Invalid label in file: {label_path}")

In [10]:
extract_features(text_features, text_clf_targets, text_reg_targets)

2024-10-19 14:36:09,711 INFO: 1 batches, avg len: 70.3


2024-10-19 14:36:13,364 INFO: 1 batches, avg len: 45.7
2024-10-19 14:36:15,927 INFO: 1 batches, avg len: 40.7
2024-10-19 14:36:18,482 INFO: 1 batches, avg len: 22.0
2024-10-19 14:36:19,751 INFO: 1 batches, avg len: 24.3
2024-10-19 14:36:21,659 INFO: 1 batches, avg len: 69.0
2024-10-19 14:36:25,517 INFO: 1 batches, avg len: 74.0
2024-10-19 14:36:30,582 INFO: 1 batches, avg len: 11.3
2024-10-19 14:36:31,304 INFO: 1 batches, avg len: 10.0
2024-10-19 14:36:31,927 INFO: 1 batches, avg len: 31.7
2024-10-19 14:36:34,622 INFO: 1 batches, avg len: 150.0
2024-10-19 14:36:45,440 INFO: 1 batches, avg len: 197.7
2024-10-19 14:36:57,078 INFO: 1 batches, avg len: 37.3
2024-10-19 14:36:59,967 INFO: 1 batches, avg len: 87.7
2024-10-19 14:37:05,764 INFO: 1 batches, avg len: 21.3
2024-10-19 14:37:07,009 INFO: 1 batches, avg len: 61.3
2024-10-19 14:37:10,392 INFO: 1 batches, avg len: 25.0
2024-10-19 14:37:12,096 INFO: 1 batches, avg len: 39.0
2024-10-19 14:37:14,437 INFO: 1 batches, avg len: 19.7
2024-10-

In [11]:
print("Saving npz file locally...")
np.savez('./Features/TextWhole/whole_samples_reg_avg.npz', text_features)
np.savez('./Features/TextWhole/whole_labels_reg_avg.npz', text_reg_targets)
np.savez('./Features/TextWhole/whole_samples_clf_avg.npz', text_features)
np.savez('./Features/TextWhole/whole_labels_clf_avg.npz', text_clf_targets)

Saving npz file locally...
